In [1]:
import pandas as pd
import numpy as np
import os
import json
from tqdm.notebook import tqdm
import shutil

In [2]:
import torch
print(torch.__version__)

1.10.0+cu111


In [3]:
!nvidia-smi

Thu Mar  3 10:54:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:


!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html
     |████████████████████████████████| 46.0 MB 9.4 MB/s 
     |████████████████████████████████| 190 kB 5.2 MB/s 


In [5]:

!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .


Cloning into 'mmdetection'...
remote: Enumerating objects: 23446, done.
remote: Total 23446 (delta 0), reused 0 (delta 0), pack-reused 23446
Receiving objects: 100% (23446/23446), 26.05 MiB | 30.88 MiB/s, done.
Resolving deltas: 100% (16388/16388), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Running setup.py develop for mmdet


In [6]:
!python setup.py develop

running develop
running egg_info
writing mmdet.egg-info/PKG-INFO
writing dependency_links to mmdet.egg-info/dependency_links.txt
writing requirements to mmdet.egg-info/requires.txt
writing top-level names to mmdet.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'mmdet.egg-info/SOURCES.txt'
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:381: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/mmdet.egg-link (link to .)
mmdet 2.22.0 is already the active version in easy-install.pth

Installed /content/mmdetection
Processing dependencies for mmdet==2.22.0
Searching for terminaltables==3.1.10
Best match: terminaltables 3.1.10
Adding terminaltables 3.1.10 to easy-install.pth file

Using /usr/

In [7]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu111 True
2.22.0
11.1
GCC 7.3


## **Old**

In [ ]:
with open('/content/data/train_segmentation/annotations.json') as f:
    data = json.load(f)


In [ ]:
val = dict(categories=[{'id': 1, 'name': 'text', 'supercategory': ''}], 
           images=[],
           annotations=[])
train = dict(categories=[{'id': 1, 'name': 'text', 'supercategory': ''}], 
           images=[],
           annotations=[])
for index, d in enumerate(data['images']):
  print(f"{index}/{len(data['images'])}")
  if index % 11 == 0:
    val['images'].append(d)
    for i in data['annotations']:
      if i['image_id'] == d['id']:
        val['annotations'].append(i)
  else:
    train['images'].append(d)
    for i in data['annotations']:
      if i['image_id'] == d['id']:
        train['annotations'].append(i)

    

In [ ]:
path2save = 'train'
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
coco = COCO('/content/data/train_segmentation/annotations.json')
cat_ids = coco.getCatIds()


os.makedirs('convertor/seg')
for i in tqdm(train['images']):
  name = i['file_name']
  if not os.path.exists('convertor/{}/images/'.format(path2save)):
                os.makedirs('convertor/{}/images/'.format(path2save))
  if not os.path.exists('convertor/seg/{}'.format(path2save)):
                os.makedirs('convertor/seg/{}'.format(path2save))
  shutil.copy("/content/data/train_segmentation/images/{}".format(name),'convertor/{}/images/'.format(path2save,name))

  img = coco.imgs[i["id"]]
  anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
  anns = coco.loadAnns(anns_ids)
  anns_img = np.zeros((img['height'],img['width']))
  for ann in anns:
      anns_img = np.maximum(anns_img,coco.annToMask(ann)*ann['category_id'])
  cv2.imwrite(f"convertor/seg/{path2save}/{name}", anns_img * 255)

with open(('convertor/{}/annotations.json'.format(path2save)), 'w', encoding='utf8') as f:
  json.dump(train, f, ensure_ascii=False, indent=4)

path2save = 'val'
for i in tqdm(val['images']):
  name = i['file_name']
  if not os.path.exists('convertor/{}/images/'.format(path2save)):
                os.makedirs('convertor/{}/images/'.format(path2save))
  if not os.path.exists('convertor/seg/{}'.format(path2save)):
                os.makedirs('convertor/seg/{}'.format(path2save))
  shutil.copy("/content/data/train_segmentation/images/{}".format(name),'convertor/{}/images/'.format(path2save,name))

  img = coco.imgs[i["id"]]
  anns_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids, iscrowd=None)
  anns = coco.loadAnns(anns_ids)
  anns_img = np.zeros((img['height'],img['width']))
  for ann in anns:
      anns_img = np.maximum(anns_img,coco.annToMask(ann)*ann['category_id'])
  cv2.imwrite(f"convertor/seg/{path2save}/{name}", anns_img * 255)
  
with open(('convertor/{}/annotations.json'.format(path2save)), 'w', encoding='utf8') as f:
  json.dump(val, f, ensure_ascii=False, indent=4)


In [ ]:

import numpy
import cv2


def decodeSeg(mask, segmentations):
    """
    Draw segmentation
    """
    pts = [
        numpy
            .array(anno)
            .reshape(-1, 2)
            .round()
            .astype(int)
        for anno in segmentations
    ]
    mask = cv2.fillPoly(mask, pts, 1)

    return mask


def annotation2binarymask(annotations):
    mask = numpy.zeros((h, w), numpy.uint8)
    for annotation in annotations:
        #segmentations = annotation['segmentation']
        segmentations = annotations
        if isinstance(segmentations, list): # segmentation
            mask = decodeSeg(mask, segmentations)
    return mask

In [ ]:
aa = annotation2binarymask(seg)

In [ ]:
cv2.imwrite(f"img.png", aa * 255)

True

In [ ]:
pwd

'/content/mmdetection'

# **Train**

In [39]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/НТИ_ИИ/data_team/detectrors_data.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/mmdetection")

In [12]:
import os
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import mmcv
import re
import xml.etree.ElementTree as ET
from typing import Dict, List

In [35]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/detectors/detectors_cascade_rcnn_r50_1x_coco/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth \
      -O checkpoints/model_2.pth

--2022-03-03 11:25:37--  https://download.openmmlab.com/mmdetection/v2.0/detectors/detectors_cascade_rcnn_r50_1x_coco/detectors_cascade_rcnn_r50_1x_coco-32a10ba0.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.245.16.66
Connecting to download.openmmlab.com (download.openmmlab.com)|47.245.16.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 496633663 (474M) [application/octet-stream]
Saving to: ‘checkpoints/model_2.pth’

checkpoints/model_2 100%[===================>] 473.63M  7.06MB/s    in 78s     

2022-03-03 11:26:56 (6.05 MB/s) - ‘checkpoints/model_2.pth’ saved [496633663/496633663]



In [34]:
%cd mmdetection

/content/mmdetection


In [40]:
with open("/content/labels.txt", "w") as f:
  f.write("text")

In [48]:
from mmcv import Config
with open("labels.txt", "w") as f:
      f.write("text")
cfg = Config.fromfile('configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')
cfg.dataset_type = 'CocoDataset'
cfg.classes = 'labels.txt'
cfg.data_root = 'mmdetection/converter'
#cfg.model.roi_head.bbox_head.num_classes = 1
#cfg.model.roi_head.mask_head.num_classes = 1
#cfg.model.bbox_head.num_classes = 1
#cfg.model.mask_head.num_classes = 1
#cfg.model.segm_head.num_classes = 1
for i in range(len(cfg.model.roi_head.bbox_head)):  
  cfg.model.roi_head.bbox_head[i].num_classes = 1
# for i in range(len(cfg.model.roi_head.mask_head)):  
#   cfg.model.roi_head.mask_head[i].num_classes = 1
cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = 'convertor/val'
cfg.data.test.ann_file = 'annotations.json'
cfg.data.test.img_prefix = 'images'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = 'convertor/train'
cfg.data.train.ann_file = 'annotations.json'
cfg.data.train.img_prefix = 'images'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.classes = 'labels.txt'
cfg.data.val.data_root = 'convertor/val'
cfg.data.val.ann_file = 'annotations.json'
cfg.data.val.img_prefix = 'images'

cfg.lr_config.step = 7
cfg.optimizer.lr = 0.01 / 8
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 2
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 6
cfg.checkpoint_config.interval = 3
cfg.log_config.interval = 100
cfg.runner.max_epochs = 30
cfg.model.test_cfg.max_per_img = 1000
cfg.model.test_cfg.rcnn.max_per_img = 1000
cfg.work_dir = '/content/drive/MyDrive/НТИ_ИИ/team/semeka_models'
#cfg.model.train_cfg.max_per_img = 1000
#cfg.model.test_cfg.rcnn.mask_thr_binary = 0.3

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.load_from = 'checkpoints/model_2.pth'
#print(f'Config:\n{cfg.pretty_text}')

In [37]:
cfg.palette

[(255, 255, 255)]

In [36]:
for i in cfg.keys():
  if 'palette' in str(cfg[i]).lower():
    print(i)


In [49]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('val_cfg'))

model.CLASSES = datasets[0].CLASSES
#model.PALETTE = [(255, 255, 255)]
#datasets[0].PALETTE = [(255, 255, 255)]
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))


loading annotations into memory...
Done (t=4.01s)
creating index...
index created!


In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...


2022-03-03 11:30:53,173 - mmdet - INFO - load checkpoint from local path: checkpoints/model_2.pth


Done (t=0.28s)
creating index...
index created!


2022-03-03 11:30:53,698 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([2]).
size mismatch for roi_head.bbox_head.2.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([2, 1024]).
size mismatch for roi_head.bbox_head.2.fc_cls.bias: copying 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 85/85, 4.2 task/s, elapsed: 20s, ETA:     0s

...

2022-03-03 14:08:36,899 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=52.67s).
Accumulating evaluation results...


2022-03-03 14:09:31,031 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.408
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.807
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.658
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.374
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.689
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.434
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.599
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.599
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.35s).


2022-03-03 14:11:04,709 - mmdet - INFO - Epoch [13][100/847]	lr: 1.250e-04, eta: 3:38:41, time: 0.936, data_time: 0.029, memory: 8220, loss_rpn_cls: 0.0220, loss_rpn_bbox: 0.0429, s0.loss_cls: 0.0875, s0.acc: 96.3730, s0.loss_bbox: 0.0792, s1.loss_cls: 0.0396, s1.acc: 96.8401, s1.loss_bbox: 0.0729, s2.loss_cls: 0.0237, s2.acc: 96.2566, s2.loss_bbox: 0.0429, loss: 0.4108, grad_norm: 3.3607
2022-03-03 14:12:35,956 - mmdet - INFO - Epoch [13][200/847]	lr: 1.250e-04, eta: 3:37:21, time: 0.912, data_time: 0.004, memory: 8220, loss_rpn_cls: 0.0221, loss_rpn_bbox: 0.0432, s0.loss_cls: 0.0870, s0.acc: 96.3496, s0.loss_bbox: 0.0797, s1.loss_cls: 0.0383, s1.acc: 96.9299, s1.loss_bbox: 0.0738, s2.loss_cls: 0.0236, s2.acc: 96.1167, s2.loss_bbox: 0.0413, loss: 0.4089, grad_norm: 3.3396
2022-03-03 14:14:07,086 - mmdet - INFO - Epoch [13][300/847]	lr: 1.250e-04, eta: 3:36:01, time: 0.911, data_time: 0.004, memory: 8220, loss_rpn_cls: 0.0233, loss_rpn_bbox: 0.0455, s0.loss_cls: 0.0854, s0.acc: 96.4688

In [ ]:
!pwd

/content/mmdetection


In [31]:
from mmdet.apis import set_random_seed
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
class SegmentationModel:
  def __init__(self, path2model, path2config='detectors/cascade_rcnn_r50_sac_1x_coco.py'):
    with open("labels.txt", "w") as f:
      f.write("text")
    self.cfg = Config.fromfile('mmdetection/configs/' + path2config)
    self.cfg.dataset_type = 'CocoDataset'
    self.cfg.classes = 'labels.txt'
    self.cfg.data_root = 'mmdetection/converter'
    #cfg.model.roi_head.bbox_head.num_classes = 1
    #cfg.model.roi_head.mask_head.num_classes = 1
    #cfg.model.bbox_head.num_classes = 1
    #cfg.model.mask_head.num_classes = 1
    #cfg.model.segm_head.num_classes = 1
    for i in range(len(self.cfg.model.roi_head.bbox_head)):  
      self.cfg.model.roi_head.bbox_head[i].num_classes = 1
    # for i in range(len(cfg.model.roi_head.mask_head)):  
    #   cfg.model.roi_head.mask_head[i].num_classes = 1
    self.cfg.data.test.type = 'CocoDataset'
    self.cfg.data.test.classes = 'labels.txt'
    self.cfg.data.test.data_root = 'mmdetection/convertor/val'
    self.cfg.data.test.ann_file = 'annotations.json'
    self.cfg.data.test.img_prefix = 'images'

    self.cfg.data.train.type = 'CocoDataset'
    self.cfg.data.train.data_root = 'mmdetection/convertor/train'
    self.cfg.data.train.ann_file = 'annotations.json'
    self.cfg.data.train.img_prefix = 'images'
    self.cfg.data.train.classes = 'labels.txt'

    self.cfg.data.val.type = 'CocoDataset'
    self.cfg.data.val.classes = 'labels.txt'
    self.cfg.data.val.data_root = 'mmdetection/convertor/val'
    self.cfg.data.val.ann_file = 'annotations.json'
    self.cfg.data.val.img_prefix = 'images'

    self.cfg.lr_config.step = 7
    self.cfg.data.samples_per_gpu = 1
    self.cfg.data.workers_per_gpu = 1
    self.cfg.evaluation.metric = 'bbox'
    self.cfg.evaluation.interval = 12
    self.cfg.checkpoint_config.interval = 3
    self.cfg.log_config.interval = 100
    self.cfg.runner.max_epochs = 30
    self.cfg.model.test_cfg.max_per_img = 1000
    self.cfg.model.test_cfg.rcnn.max_per_img = 1000
    #cfg.model.train_cfg.max_per_img = 1000
    #cfg.model.test_cfg.rcnn.mask_thr_binary = 0.3

    self.cfg.seed = 0
    set_random_seed(0, deterministic=False)
    self.cfg.gpu_ids = range(1)
    self.cfg.load_from = path2model
    self.model = init_detector(self.cfg, path2model, device='cuda')
    #self.model = build_detector(self.cfg.model, train_cfg=self.cfg.get('train_cfg'), test_cfg=self.cfg.get('val_cfg'))
    self.model.cfg = self.cfg
    self.model.CLASSES = ['text']
  def predict(self, imgs):
    data = [mmcv.imread(i) for i in imgs]  
    result = inference_detector(self.model, data)
    output = []
    for img in result:
      out_img = []
      for box in img[0]:
        out = []
        #print(len(box))
        left, top, right, bottom, c = box
        out.append([[left, top]])
        out.append([[right, top]])
        out.append([[right, bottom]])
        out.append([[left, bottom]])
        out_img.append(out)
      output.append(out_img)
    return result[0], model

In [9]:
%cd ../

/content


In [32]:
dd = SegmentationModel(path2model='/content/drive/MyDrive/НТИ_ИИ/team/semeka_models/epoch_12.pth')

load checkpoint from local path: /content/drive/MyDrive/НТИ_ИИ/team/semeka_models/epoch_12.pth


In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(img)

In [ ]:
left, top, right, bottom, c = list(map(int, result[0][0][0]))
out = []
out.append([[left, top]])
out.append([[right, top]])
out.append([[right, bottom]])
out.append([[left, bottom]])


In [ ]:
out

[[[2291, 1660]], [[2823, 1660]], [[2823, 1778]], [[2291, 1778]]]

In [ ]:
img = mmcv.imread('/content/105_0.JPG')

In [33]:
result= dd.predict(['/content/105_0.JPG', '/content/107_7.jpg', '/content/0_2_eng.jpg'])
img = mmcv.imread('/content/105_0.JPG')
show_result_pyplot(model, img, result[0], score_thr=0.3)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
result

In [20]:
img = mmcv.imread('/content/107_7.jpg')
show_result_pyplot(model, img, result[1], score_thr=0.3)

Output hidden; open in https://colab.research.google.com to view.

In [21]:
img = mmcv.imread('/content/0_2_eng.jpg')
show_result_pyplot(model, img, result[2], score_thr=0.3)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img = mmcv.imread('/content/data/train_segmentation/images/100_0.JPG')

In [ ]:
from mmdet.core import encode_mask_results
## bbox -> result[0] and mask -> result[1] for single image
encode_result = (result[0], encode_mask_results(result[1]))
encode_result # containing bbox and segm

In [ ]:
def polygonFromMask(maskedArr): # https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py

        contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        segmentation = []
        for contour in contours:
            # Valid polygons have >= 6 coordinates (3 points)
            if contour.size >= 6:
                segmentation.append(contour.flatten().tolist())
        RLEs = mask.frPyObjects(segmentation, maskedArr.shape[0], maskedArr.shape[1])
        RLE = mask.merge(RLEs)
        # RLE = mask.encode(np.asfortranarray(maskedArr))
        area = mask.area(RLE)
        [x, y, w, h] = cv2.boundingRect(maskedArr)

        return segmentation[0] #, [x, y, w, h], area

import pycocotools.mask as mask
maskedArr = mask.decode(compactRLESegmentation)

In [ ]:
from mmcv import Config
cfg = Config.fromfile('configs/detectors/htc_r50_sac_1x_coco.py')
from mmdet.apis import set_random_seed
print(cfg.model.keys())
cfg.dataset_type = 'CocoDataset'
#cfg.classes = '/content/labels.txt'
cfg.data_root = '/content/mmdetection/converter'
#cfg.model.roi_head.bbox_head.num_classes = 1
#cfg.model.roi_head.mask_head.num_classes = 1
#cfg.model.bbox_head.num_classes = 1
# cfg.model.roi_head.mask_head.num_classes = 1
# cfg.model.roi_head.glbctx_head.num_classes = 1
cfg.classes = ('text',)
cfg.palette = [(255, 255, 255)]

for i in range(len(cfg.model.roi_head.bbox_head)):  
  cfg.model.roi_head.bbox_head[i].num_classes = 1
for i in range(len(cfg.model.roi_head.mask_head)):  
  cfg.model.roi_head.mask_head[i].num_classes = 1
cfg.model.roi_head.semantic_head.num_classes = 1
cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = '/content/labels.txt'
#cfg.data.test.data_root = '/content/mmdetection/convertor'
cfg.data.test.ann_file = 'convertor/val/annotations.json'
cfg.data.test.img_prefix = 'convertor/val/images'
cfg.data.test.seg_prefix = 'convertor/seg/val'

cfg.data.train.type = 'CocoDataset'
#cfg.data.train.data_root = '/content/mmdetection/convertor'
cfg.data.train.ann_file = 'convertor/train/annotations.json'
cfg.data.train.img_prefix = 'convertor/train/images'
cfg.data.train.classes = '/content/labels.txt'
cfg.data.train.seg_prefix = 'convertor/seg/train'
#cfg.data.train.pipeline[1].with_seg = False
#cfg.data.train.pipeline[8].keys = ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']

cfg.data.val.type = 'CocoDataset'
cfg.data.val.classes = '/content/labels.txt'
#cfg.data.val.data_root = '/content/mmdetection/convertor/val'
cfg.data.val.ann_file = 'convertor/val/annotations.json'
cfg.data.val.img_prefix = 'convertor/val/images'
cfg.data.val.seg_prefix = 'convertor/seg/val'


#cfg.load_from = '../input/retinanet/retinanet_r101_fpn_1x_coco_20200130-7a93545f (1).pth'

cfg.work_dir = '/content/drive/MyDrive/НТИ_ИИ/team/semeka_models'

# The original learning rate (LR) is set for 8-GPU training.
# You divide it by 8 since you only use one GPU with Kaggle.
cfg.optimizer.lr = 0.01 / 8
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

#cfg.lr_config.policy = 'step'
cfg.lr_config.step = 7
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 6
cfg.checkpoint_config.interval = 3
cfg.log_config.interval = 100
cfg.runner.max_epochs = 20
cfg.model.test_cfg.max_per_img = 1000
cfg.model.test_cfg.rcnn.max_per_img = 1000
#cfg.model.neck.fp16_enabled = True
#cfg.model.train_cfg.max_per_img = 1000
#cfg.model.test_cfg.rcnn.mask_thr_binary = 0.3

cfg.seed = 0
set_random_seed(1, deterministic=False)
cfg.gpu_ids = range(1)
cfg.load_from = 'checkpoints/model_1.pth'
#print(f'Config:\n{cfg.pretty_text}')